In [3]:
import numpy as np
import pandas as pd
import quaternion
import re
import datetime
import os
import glob

def normalize_vector(v):
        norm_v=v/np.linalg.norm(v)
        return norm_v

class read():
    def pdb(filename):
        inputfile = str(filename)
        outputfile = inputfile.strip(".pdb")
        with open(inputfile,'r') as fp:
            content = fp.readlines()
            linesnumber = len(content)
        lines = [] 
        with open(outputfile+'.txt','w') as fp_w:
            for i in range (linesnumber):
                values = content[i].split() if content[i].strip() != '' else None
                if values == None:
                    continue
                if (values[0]=='ATOM' or values[0] == 'HETATM'):
                    value1 = values[2] #atom_label
                    value2 = values[3] #res_name
                    value3 = float(values[5]) #x
                    value4 = float(values[6]) #y
                    value5 = float(values[7]) #z
                    value6 = values[10] #atom_note
                    value7 = int(values[4])
                    newline = "%7s%7s%5d%8.3f%8.3f%8.3f%7s" % (
                        value1, value2, value7,value3, value4, value5, value6
                        )
        
                    lines.append(newline+'\n')
            fp_w.writelines(lines)
        data = pd.read_csv(outputfile+'.txt',sep='\s+',names=['Atom_label','Residue','Res_number','x','y','z','Note'])
        return data
    
class output():
    def __init__(self,name,outpdb=True, outxyz=True, outgro=True):
        self.output = str(name)
        #self.begin_lines = useless_atom_number
        if outxyz:
            output.outxyz(self)
        if outgro:
            output.outgro(self)
        if outpdb:
            output.outpdb(self)

        '''output'''
    def outgro(self):
        with open(self.output+'.txt', 'r') as f:
            # Read the lines from the file
            lines = f.readlines()
            atoms_number = len(lines)
        newgro = []
        with open(self.output+'.gro', 'w') as fp:
            newgro.append("generated by MOF_BUILD"+'\n'+str(atoms_number)+'\n')
            for i in range (atoms_number):
                values = lines[i].split()
                value_atom_number = int(i+1) #atom_number
                value_label = values[0] #atom_label
                value_resname = values[1] #residue_name
                value_resnumber = int(float(values[2])) #residue number
                value_x = float(values[4])/10 #x      
                value_y = float(values[5])/10 #y
                value_z = float(values[6])/10 #z
                formatted_line = "%5d%-5s%5s%5d%8.3f%8.3f%8.3f" % (
                            value_resnumber, value_resname, value_label, value_atom_number, value_x, value_y, value_z) 
                newgro.append(formatted_line+'\n')        
            tail = '5 5 5 \n'
            newgro.append(tail)
            fp.writelines(newgro)

    def outxyz(self):
        with open(self.output+'.txt', 'r') as f:
            lines = f.readlines()
            atoms_number = len(lines)
        newxyz = []
        with open(self.output+'.xyz', 'w') as fp:
            newxyz.append(str(atoms_number)+'\n'+"generated by MOF_BUILD"+'\n')
            for i in range (atoms_number):
                values = lines[i].split()
                value_label = values[0] #atom_label
                value_label = re.sub(r'\d', '', value_label)
                value_x = float(values[4]) #x      
                value_y = float(values[5]) #y
                value_z = float(values[6]) #z
                formatted_line = "%-5s%8.3f%8.3f%8.3f" % (
                            value_label, value_x, value_y, value_z
                )        
                newxyz.append(formatted_line+'\n')        
            fp.writelines(newxyz)

    def outpdb(self):
        with open(self.output+'.txt', 'r') as f:
        # Read the lines from the file
            lines = f.readlines()
            atoms_number = len(lines)

        newpdb = []
        with open(self.output+'.txt', 'w') as fp:
            # Iterate over each line in the input file
            for i in range (atoms_number):
                # Split the line into individual values (assuming they are separated by spaces)
                values = lines[i].split()
                # Extract values based on their positions in the format string
                value1 = 'ATOM'
                value2 = int(i+1)
                value3 = values[0] #label
                value4 = values[1] #residue
                value5 = int(float(values[2])) #residue number
                value6 = float(values[4]) #x      
                value7 = float(values[5]) #y
                value8 = float(values[6]) #z
                value9 = '1.00'
                value10 = '0.00'
                value11 = values[3] #note
                # Format the values using the specified format string
                formatted_line = "%-6s%5d%5s%4s%10d%8.3f%8.3f%8.3f%6s%6s%12s" % (
                            value1, value2, value3, value4, value5, value6, value7, value8,value9,value10,value11
                )        
                lines[i] = formatted_line+'\n'        
            fp.writelines(lines)

        with open(self.output+'.pdb', 'w') as fp:
            # Iterate over each line in the input file
            newpdb.append("generated by MOF_BUILD"+'\n')
            newpdb.append(lines[0])
            
            for i in range (1,len(lines)):
                lastline = lines[i-1]
                thisline = lines[i]
                # Split the line into individual values (assuming they are separated by spaces)
                old_residue_number = lastline.split()[4]
                new_residue_number = thisline.split()[4]
                
                if(old_residue_number != new_residue_number):
                    newline ='TER'+'\n'+thisline
                    newpdb.append(newline)
                else:
                    newline = thisline
                    newpdb.append(newline)
            fp.writelines(newpdb)



In [34]:
#node = read.pdb('PCN222_lib_nodeZr.pdb')
node = read.pdb('node_lib.pdb')
#print(node.iloc[30,:],node.iloc[57,:])
array = node.loc[:, ['x','y','z']].to_numpy()
center = 0.5*(array[47]+array[46])
new_node = array-center


In [21]:
import sys
sys.path.append("..")

In [35]:
from MOF_build.functions.rotate import rotate
any_tdx, any_tdy, any_tdz = (
np.array([1, 0, 0]),
np.array([0, 1, 0]),
np.array([0, 1, 2]))
dx, dy, dz = np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])
carte_basis = np.vstack([dx, dy, dz])
    # for uio66 fcb
tric_basis = rotate.coordinate_transfer(any_tdx, any_tdy, any_tdz, carte_basis)
#v1_file = new_node[9]-new_node[0]
#v2_file = new_node[27]-new_node[18]
v1_file = new_node[47]-new_node[46]
v2_file = new_node[40]-new_node[43]
v1_frame = tric_basis[2]-tric_basis[1]
v2_frame = tric_basis[2]+tric_basis[1]
rotated_node = rotate.rotate_twice_linker(node,center,v1_file,v1_frame,v2_file,v2_frame)


In [36]:
node.loc[:, ['x','y','z']] = rotated_node
newdf = pd.concat([node.iloc[:,0:3],node.iloc[:,6],node.iloc[:,3:6]],axis=1, join = 'outer')
newdf.to_csv('PCNZRRnodeOO.txt',header=None,sep='\t',index=False)
output('PCNZRRnodeOO',True,False,False)

In [26]:
justnode = read.pdb('ZRRnode.pdb')
justnode.iloc[:,2]=1 #resnumber
print(justnode)
df = pd.concat([justnode.iloc[:,0:3],justnode.iloc[:,6],justnode.iloc[:,3:6]],axis=1, join = 'outer')
df.to_csv('lib_ZRnode.txt',header=None,sep='\t',index=False)
output('lib_ZRnode',True,False,False)

   Atom_label Residue  Res_number      x      y      z Note
0          O1     OMM           1 -1.305 -0.351 -1.084    O
1          O1     OMM           1  1.534 -0.246 -0.791    O
2          O1     OMM           1 -0.071  1.707  0.517    O
3          O1     OMM           1 -0.113 -1.044  1.405    O
4          O2     OHH           1  0.181  1.314 -1.775    O
..        ...     ...         ...    ...    ...    ...  ...
61        DD4     ZRR           1 -0.155  1.315  1.379    D
62        DD5     ZRR           1  0.272  1.518  2.623    D
63        DD6     ZRR           1  0.594  0.553  1.776    D
64        DD7     ZRR           1  0.257  0.400  3.056    D
65        DD8     ZRR           1 -0.194 -0.234  1.888    D

[66 rows x 7 columns]


In [28]:
df1 = pd.read_csv('example_MOF_metal.gro',sep='\s+',names=['Atom_label','Residue','Res_number','Note','x','y','z'])
df['z'] = df['z'].astype(float)
df1.to_csv('lib1_ZRnode.txt',header=None,sep='\t',index=False)
output('lib1_ZRnode',True,False,False)

In [137]:
pre_dummy_pdb = read.pdb('PCNZRRnodeOO.pdb')

atom_type = []
for i in pre_dummy_pdb['Atom_label']:
    atom_type.append(re.sub(r"\d", "", i))
pre_dummy_pdb.insert(7, "atom_type", atom_type, True)

df_Zr =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='Zr'].reset_index()
df_O =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='O'].reset_index()
O_positions = df_O.loc[:, ['x','y','z']].to_numpy()
df_dummy = read.pdb("dummy_template.pdb")
for i in range(df_Zr.shape[0]):
    distance_O = []
    distance_Osort = []
    pointZR = df_Zr.loc[i, ['x','y','z']].to_numpy()
    for j in range(len(O_positions)):
        distance_O.append(np.linalg.norm(pointZR-O_positions[j]) )
        distance_Osort.append(np.linalg.norm(pointZR-O_positions[j]) )
    distance_Osort.sort()
    threshhold = distance_Osort[8]
    indices = [index for index, num in enumerate(distance_O) if num < threshhold]
    #print(indices,distance_O)
    dummy_atoms = np.zeros((8,3))
    row =0
    for k in indices:
        newD = pointZR+(O_positions[k]-pointZR)/np.linalg.norm(pointZR-O_positions[i])
        dummy_atoms[row,:]=newD
        row +=1
    print(pointZR,'\n')
    dummy_atoms=np.round(dummy_atoms,3)
    df_dummy.loc[0, ['x','y','z']]=pointZR
    df_dummy.loc[1:, ['x', 'y', 'z']] = dummy_atoms
    if i <1:
        df_add_dummy = pd.concat([read.pdb('PCNZRRnodeOO.pdb'),df_dummy],ignore_index=True)
    else:
        df_add_dummy = pd.concat([df_add_dummy,df_dummy],ignore_index=True)

#output('0adddummy',True,False,False)


[0.265 -1.303 -2.108] 

[2.477 0.142 0.23] 

[-2.477 -0.142 -0.23] 

[-0.265 1.303 2.108] 

[-0.0 2.091 -1.293] 

[-0.0 -2.091 1.293] 



In [138]:

df_add_dummy.insert(3, "note", df_add_dummy['Note'], True)
df_add_dummy.to_csv('0adddummy.txt',header=None,sep='\t',index=False)
#
output('0adddummy',True,False,False)